In [1]:
import pandas as pd
import numpy as np
import re

from elder_care.config import BLD
from elder_care.config import SRC

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
data = pd.read_csv(BLD / "data" / "estimation_data.csv")

In [4]:
data.shape

(6812, 383)

In [22]:
data[["mergeid", "care", "lagged_care"]].head(20)

mergeid  care  lagged_care
0   DE-000132-01   0.0          NaN
1   DE-000132-01   0.0          0.0
2   DE-000132-01   0.0          0.0
3   DE-000132-01   0.0          0.0
4   DE-000132-01   0.0          0.0
5   DE-000554-01   0.0          NaN
6   DE-000554-01   0.0          0.0
7   DE-000802-01   0.0          NaN
8   DE-000802-01   0.0          0.0
9   DE-000802-01   0.0          0.0
10  DE-001237-02   0.0          NaN
11  DE-001237-02   0.0          0.0
12  DE-001237-02   0.0          0.0
13  DE-001350-01   0.0          NaN
14  DE-001381-01   0.0          NaN
15  DE-002106-02   1.0          NaN
16  DE-002106-02   0.0          1.0
17  DE-002173-02   1.0          NaN
18  DE-002173-02   1.0          1.0
19  DE-002173-02   0.0          1.0

## Transition from caregiving to non-caregiving

In [7]:
data["care"].unique()

array([ 0.,  1., nan])

In [ ]:
dat = dat.sort_values(by=["mergeid", "int_year"], ascending=[True, True])
dat["lagged_care"] = dat.groupby("mergeid")["care"].shift(1)
dat["care_experience"] = dat.groupby("mergeid")["lagged_care"].cumsum()

In [9]:
df = data.copy()

In [11]:
# add an arbitrary counts column for group by
df['counts'] = 1

In [13]:
# group together the combinations then unstack to get matrix
trans_matrix = df.groupby("mergeid")["care", "lagged_care"].count().unstack()

ValueError: Cannot subset columns with a tuple with more than one element. Use a list instead.

In [14]:
trans_matrix = df.groupby(["care", "lagged_care"]).count().unstack()

In [15]:
# max the columns a bit neater
trans_matrix.columns = trans_matrix.columns.droplevel()

In [16]:
trans_matrix

lagged_care   0.0  1.0   0.0  1.0   0.0  1.0   0.0  1.0   0.0  1.0   0.0  1.0  \
care                                                                            
0.0          3324  289  3324  289  3324  289  3324  289  3324  289  3324  289   
1.0           260  141   260  141   260  141   260  141   260  141   260  141   

lagged_care   0.0  1.0   0.0  1.0   0.0  1.0   0.0  1.0  0.0  1.0  0.0  1.0  \
care                                                                          
0.0          3324  289  3324  289  2910  223  2910  223   67    2   27    2   
1.0           260  141   260  141   260  141   260  141    6    1    1    2   

lagged_care  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0   0.0  1.0   0.0  1.0  \
care                                                                        
0.0           26    2  204   17   37    3    0    0  1528  186  1331  130   
1.0            1    2   17    9    5    0    0    0   237  131   136   65   

lagged_care  0.0  1.0  0.0  1.0   0.0  1.0   0.0  1.0  0.0  1.0  0.0  1.0  \
care                                                                        
0.0          516  119  190   28  1020   67  1068   98    6    0    2    0   
1.0          213  109   60   23    29   22    74   42    1    1    0    0   

lagged_care  0.0  1.0  0.0  1.0  0.0  1.0   0.0  1.0   0.0  1.0  0.0  1.0  \
care                                                                        
0.0          487  119  178   28  153   24  2333  176  2340  177  483  118   
1.0          208  108   60   23    3    4   199  114   190  116  195   98   

lagged_care  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  \
care                                                                           
0.0          178   28   27    2   27    2   27    2   27    2   27    2   27   
1.0           58   23    1    2    1    2    1    2    1    2    1    2    1   

lagged_care  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  \
care                                                                           
0.0            2   27    2   27    2   27    2   27    2   27    2   27    2   
1.0            2    1    2    1    2    1    2    1    2    1    2    1    2   

lagged_care  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0   0.0  1.0   0.0  1.0  \
care                                                                        
0.0           27    2   27    2    5    2   27    2  1602   73  2893  220   
1.0            1    2    1    2    0    0    1    2   130   89   260  141   

lagged_care  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  \
care                                                                           
0.0          895   39  895   39  895   39  895   39  895   39  895   39  895   
1.0           78   43   78   43   78   43   78   43   78   43   78   43   78   

lagged_care  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0   0.0  1.0  0.0  1.0  \
care                                                                            
0.0           39  895   39  895   39  895   39  257   12  2141  113  666   42   
1.0           43   78   43   78   43   78   43   25   16   258  139  246  130   

lagged_care  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  \
care                                                                           
0.0          177   10   43    1  551   40  141   10   31    1  666   42  177   
1.0           88   48   29   13  193  115   68   42   20   12  246  130   88   

lagged_care  1.0  0.0  1.0  0.0  1.0  0.0  1.0   0.0  1.0  0.0  1.0  0.0  1.0  \
care                                                                            
0.0           10   43    1  666   42  177   10  1945   96  141    7  141    7   
1.0           48   29   13  246  130   88   48   223  118   25   20   25   20   

lagged_care  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  \
care                                                                           
0.0          141    7  141  

In [19]:
df = df.sort_values(by=['mergeid', 'int_year'])

# Create a transition matrix using crosstab
transition_matrix = pd.crosstab(df['lagged_care'], df['care'], normalize='index')

transition_matrix

care              0.0       1.0
lagged_care                    
0.0          0.927455  0.072545
1.0          0.672093  0.327907

In [23]:
# Create a transition matrix using crosstab
transition_matrix = pd.crosstab(df['lagged_care'], df['care'], normalize=False)

transition_matrix

care          0.0  1.0
lagged_care           
0.0          3324  260
1.0           289  141